# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,22.20,67,75,2.57,US,1717977541
1,1,avarua,-21.2078,-159.7750,25.03,53,75,2.06,CK,1717977632
2,2,emerald,-23.5333,148.1667,18.52,46,0,4.14,AU,1717977632
3,3,itoman,26.1247,127.6694,27.27,80,75,1.54,JP,1717977633
4,4,albany,42.6001,-73.9662,18.27,69,20,3.34,US,1717977412


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", scale = 0.5, color = "City")

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
clean_city_data = city_data_df.dropna()

new_city_df = clean_city_data[(clean_city_data['Max Temp'] >= 21) & (clean_city_data['Max Temp'] <= 27) & (clean_city_data['Wind Speed'] < 4.5) & (clean_city_data['Cloudiness'] == 0)]


new_city_df = new_city_df[(new_city_df[['Wind Speed']] != 0).all(axis=1)]


new_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
84,84,belgrade,44.8040,20.4651,24.63,68,0,2.57,RS,1717977655
159,159,warrensburg,38.7628,-93.7361,25.94,47,0,2.06,US,1717977663
218,218,vilhena,-12.7406,-60.1458,21.12,73,0,2.57,BR,1717977412
265,265,tamanrasset,22.7850,5.5228,26.95,12,0,2.06,DZ,1717977688
308,308,brambleton,38.9821,-77.5386,24.31,49,0,2.57,US,1717977560
318,318,zahedan,29.4963,60.8629,23.95,20,0,2.06,IR,1717977695
373,373,al burayqah,30.4062,19.5739,23.55,67,0,3.45,LY,1717977701
499,499,urumqi,43.8010,87.6005,23.53,33,0,3.00,CN,1717977713


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", "")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
84,belgrade,RS,44.8040,20.4651,68,
159,warrensburg,US,38.7628,-93.7361,47,
218,vilhena,BR,-12.7406,-60.1458,73,
265,tamanrasset,DZ,22.7850,5.5228,12,
308,brambleton,US,38.9821,-77.5386,49,
318,zahedan,IR,29.4963,60.8629,20,
373,al burayqah,LY,30.4062,19.5739,67,
499,urumqi,CN,43.8010,87.6005,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {  "categories": "accommodation.hotel", "apiKey": geoapify_key, "limit":20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
belgrade - nearest hotel: Хилтон Београд
warrensburg - nearest hotel: Days Inn by Wyndham Warrensburg
vilhena - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
brambleton - nearest hotel: Aloft Dulles Airport North
zahedan - nearest hotel: هتل صالح;بانک ملی
al burayqah - nearest hotel: فندق ماس ليبيا
urumqi - nearest hotel: 百时快捷酒店红山店


,City,Country,Lat,Lng,Humidity,Hotel Name
84,belgrade,RS,44.8040,20.4651,68,Хилтон Београд
159,warrensburg,US,38.7628,-93.7361,47,Days Inn by Wyndham Warrensburg
218,vilhena,BR,-12.7406,-60.1458,73,No hotel found
265,tamanrasset,DZ,22.7850,5.5228,12,فندق الأمان
308,brambleton,US,38.9821,-77.5386,49,Aloft Dulles Airport North
318,zahedan,IR,29.4963,60.8629,20,هتل صالح;بانک ملی
373,al burayqah,LY,30.4062,19.5739,67,فندق ماس ليبيا
499,urumqi,CN,43.8010,87.6005,33,百时快捷酒店红山店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", size = "Humidity", frame_width = 850,
    frame_height = 700, hover_cols = ['Hotel Name', 'Country'], scale = 0.5, color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)